# Kling O1 Image-to-Image API 使用指南

本 Notebook 演示如何通过 **七牛 AIToken** 平台调用 **Kling O1 Image (Image-to-Image)** 模型 API。

## 模型简介

Kling O1 Image 是一款强大的图像到图像生成模型，支持以下能力：

- **精确图像编辑**：基于文本提示词对输入图像进行精确修改
- **风格迁移**：将图像转换为不同的艺术风格或视觉效果
- **主体变换**：保持构图不变的情况下改变图像中的主体
- **多图参考**：支持最多 10 张参考图像，通过 `<<<image_1>>>`、`<<<image_1>>>` 等方式在提示词中引用

**API 端点**：`fal-ai/kling-image/o1

## 1. 环境准备

首先安装 `fal-client` 依赖包，并设置 API Key 和七牛 AIToken 队列服务地址。

In [ ]:
# 安装 fal-client
%pip install fal-client -q

In [ ]:
import os

# 设置七牛 AIToken 队列服务地址
os.environ["FAL_QUEUE_RUN_HOST"] = "api.qnaigc.com/queue"

# 设置 FAL_KEY 环境变量
# 你也可以在终端中通过 export FAL_KEY="xxx" 来设置
os.environ["FAL_KEY"] = os.environ.get("QINIU_API_KEY") or (_ for _ in ()).throw(ValueError("环境变量 QINIU_API_KEY 未设置"))

In [ ]:
import fal_client
from IPython.display import Image, display

## 2. 基础用法 — 队列调用

七牛 AIToken 平台使用队列模式调用 API：先提交请求（`submit`），然后轮询状态（`status`），最后获取结果（`result`）。

下面先封装一个通用的调用辅助函数，后续示例都会复用。

In [ ]:
import time

def run_fal_task(endpoint, arguments, poll_interval=5):
    """提交任务并轮询等待结果返回"""
    # 步骤 1：提交请求
    handler = fal_client.submit(endpoint, arguments=arguments)
    request_id = handler.request_id
    print(f"请求已提交，request_id: {request_id}")

    # 步骤 2：轮询任务状态
    while True:
        status = fal_client.status(endpoint, request_id, with_logs=True)
        print(f"当前状态: {type(status).__name__}")

        if isinstance(status, fal_client.Completed):
            print("任务完成!")
            break

        if hasattr(status, "logs") and status.logs:
            for log in status.logs:
                print(log["message"])

        time.sleep(poll_interval)

    # 步骤 3：获取结果
    return fal_client.result(endpoint, request_id)


# 基础调用示例
result = run_fal_task(
    "fal-ai/kling-image/o1",
    arguments={
        "prompt": "将这张图片转换为赛博朋克风格，添加霓虹灯光效果",
        "image_urls": [
            "https://aitoken-public.qnaigc.com/example/generate-video/running-man.jpg"
        ],
        "aspect_ratio": "16:9"
    },
)

# 打印返回结果
print(result)

In [ ]:
# 展示生成的图片
for img in result["images"]:
    print(f"图片 URL: {img['url']}")
    display(Image(url=img["url"], width=512))

## 3. 参数说明与高级用法

### 可配置参数一览

| 参数 | 类型 | 默认值 | 说明 |
|------|------|--------|------|
| `prompt` | string | *必填* | 图像生成的文本提示词，最多 2500 字符。通过 `<<<image_1>>>` 等引用参考图 |
| `image_urls` | list[string] | *必填* | 参考图像 URL 列表，最多 10 张 |
| `resolution` | enum | `"1K"` | 生成分辨率：`1K`（标准）、`2K`（高清）、`4K`（超高清） |
| `num_images` | int (1-9) | `1` | 生成图片数量（仅 `result_type="single"` 时有效） |
| `aspect_ratio` | enum | `"auto"` | 宽高比：`16:9`、`9:16`、`1:1`、`4:3`、`3:4`、`3:2`、`2:3`、`21:9`、`auto` |

In [ ]:
# 高级用法：使用更多参数
result_advanced = run_fal_task(
    "fal-ai/kling-image/o1",
    arguments={
        "prompt": "将  <<<image_1>>> 转换为水彩画风格，保持原始构图",
        "image_urls": [
            "https://aitoken-public.qnaigc.com/example/generate-video/running-man.jpg"
        ],
        "resolution": "2K",          # 高清分辨率
        "num_images": 2,              # 生成 2 张图片
        "aspect_ratio": "16:9",       # 宽屏比例
    },
)

# 展示结果
for i, img in enumerate(result_advanced["images"]):
    print(f"\n--- 图片 {i + 1} ---")
    print(f"URL: {img['url']}")
    display(Image(url=img["url"], width=512))

## 4. 队列模式 — 手动分步调用

上面的 `run_fal_task` 封装了完整的队列流程。如果你需要更精细的控制（例如在提交后做其他事情，稍后再取结果），可以手动分步调用。

In [ ]:
# 步骤 1：提交请求
handler = fal_client.submit(
    "fal-ai/kling-image/o1",
    arguments={
        "prompt": "将这张图片转换为油画风格",
        "image_urls": [
            "https://aitoken-public.qnaigc.com/example/generate-video/running-man.jpg"
        ],
    },
)

# 获取请求 ID，用于后续查询
request_id = handler.request_id
print(f"请求已提交，request_id: {request_id}")

In [ ]:
# 步骤 2：查询任务状态
import time

while True:
    status = fal_client.status(
        "fal-ai/kling-image/o1",
        request_id,
        with_logs=True,
    )
    print(f"当前状态: {type(status).__name__}")

    if isinstance(status, fal_client.Completed):
        print("任务完成!")
        break

    # 每 5 秒查询一次
    time.sleep(5)

In [ ]:
# 步骤 3：获取结果
result_async = fal_client.result(
    "fal-ai/kling-image/o1",
    request_id,
)

# 展示生成的图片
for img in result_async["images"]:
    print(f"图片 URL: {img['url']}")
    display(Image(url=img["url"], width=512))

## 6. Schema 参考

### Input Schema

```json
{
  "prompt": "string (必填, 最多 2500 字符)",
  "image_urls": ["string (必填, 最多 10 张)"],
  "resolution": "1K | 2K | 4K (默认 1K)",
  "num_images": "integer 1-9 (默认 1)",
  "aspect_ratio": "16:9 | 9:16 | 1:1 | 4:3 | 3:4 | 3:2 | 2:3 | 21:9 | auto (默认 auto)"
}
```

### Output Schema

```json
{
  "images": [
    {
      "url": "string (图片 URL)",
      "content_type": "string (MIME 类型)"
    }
  ]
}
```

### 更多资源

- [七牛 AIToken 文档](https://developer.qiniu.com/aitokenapi)
- [fal-client Python 文档](https://docs.fal.ai/reference/client-libraries/python/fal_client)
- [七牛 API 调用示例文档](https://apidocs.qnaigc.com)